# Perform parameter exploration

In [1]:
%load_ext autoreload
%autoreload 2

import denest
import nest
import yaml
from pathlib import Path
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from denest import ParamsTree

In [3]:
# Path to the parameter files to use
PARAMS_PATH = './data/params/tree_paths.yml'

## Override parameters

#### Load  "base" parameter files

In [7]:
base_tree = denest.load_trees(PARAMS_PATH)

2020-06-28 17:26:19,643 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-28 17:26:19,645 [denest] INFO: Finished loading parameter file paths
2020-06-28 17:26:19,647 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']


#### Define some overrides from dictionary

In [5]:
# We can override some parameters loaded from the file
overrides = [

  # Maybe change the nest kernel's settings ?
  {'kernel': {'nest_params': {'local_num_threads': 20}}},

  # Maybe change a parameter for all the projections at once ?
  {'network': {'projection_models': {'nest_params': {
      'allow_autapses': True
  }}}},

]

#### Define some overrides using the ``AutoDict`` class

The ``AutoDict`` class provides a convenient way of creating deeply nested dictionaries

In [20]:
from denest.utils.autodict import AutoDict

In [34]:
override = AutoDict({
    ('kernel', 'nest_params', 'local_num_threads'): 20,
    ('network', 'projection_models', 'nest_params', 'allow_autapses'): 20,
}).todict()

overrides = [override]

#### Load overriden parameter tree

In [35]:
# The ``denest.load_trees`` method supports overrides
overriden_tree = denest.load_trees(PARAMS_PATH, *overrides)

2020-06-28 17:36:49,078 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-28 17:36:49,079 [denest] INFO: Using 1 override tree(s)
2020-06-28 17:36:49,081 [denest] INFO: Finished loading parameter file paths
2020-06-28 17:36:49,085 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']


In [36]:
# The data defined in the overrides was inserted at the corresponding nodes

print('Base params:')
print(f"kernel nest_params: ``{base_tree.children['kernel'].nest_params}``")
print(f"root 'projection_models' nest_params: ``{base_tree.children['network'].children['projection_models'].nest_params}``")
print('...')
print('Overriden params:')
print(f"kernel nest_params: ``{overriden_tree.children['kernel'].nest_params}``")
print(f"root 'projection_models' nest_params: ``{overriden_tree.children['network'].children['projection_models'].nest_params}``")

Base params:
kernel nest_params: ``DeepChainMap({'resolution': 0.5, 'overwrite_files': True}, {})``
root 'projection_models' nest_params: ``DeepChainMap({'connection_type': 'divergent', 'mask': {'circular': {'radius': 2.0}}, 'kernel': 1.0}, {}, {})``
...
Overriden params:
kernel nest_params: ``DeepChainMap({'resolution': 0.5, 'overwrite_files': True, 'local_num_threads': 20}, {})``
root 'projection_models' nest_params: ``DeepChainMap({'connection_type': 'divergent', 'mask': {'circular': {'radius': 2.0}}, 'kernel': 1.0, 'allow_autapses': 20}, {}, {})``


#### Or merge parameter trees manually using the ``ParamsTree.merge`` method

In [37]:
# We can also use the ``ParamsTree.merge`` method

overriden_tree_2 = ParamsTree.merge(
    base_tree,
    *[
        ParamsTree(override)
        for override in overrides
    ]
)

In [38]:
print(overriden_tree_2 == overriden_tree)

True


# Perform parameter exploration

We can run multiple simulations using overrides

In [67]:
param_names = [
    'g_peak_AMPA_all_neurons',
    'weights_proj_1_AMPA',
    'weights_proj_2_GABAA',
]

# Values of the parameters
explored_values = (
    [0.1, 0.2], # 'g_peak_AMPA_all_neurons',
    [1.0, 2.0], # 'weights_proj_1_AMPA',
    [1.0, 2.0], # 'weights_proj_2_GABA_A',
)

# Position in the tree of the modified params
# Be careful not to make any mistake here!!
params_paths = {
    'g_peak_AMPA_all_neurons': ('network', 'neuron_models', 'my_neuron', 'nest_params', 'g_peak_AMPA'),
    'weights_proj_1_AMPA': ('network', 'projection_models', 'proj_1_AMPA', 'nest_params', 'weights'),
    'weights_proj_2_GABAA': ('network', 'projection_models', 'proj_2_GABAA', 'nest_params', 'weights'),
}

In [76]:
import itertools

# We save all the simulations there
MAIN_OUTPUT_DIR = Path('./data/output_param_explore')

# Run one simulation for each  combination of parameters
for param_values_combination in itertools.product(*explored_values):
    
    param_combination = dict(zip(param_names, param_values_combination))
    print(f'param combination: {param_combination}', flush=True)
    
    # Create override tree to incorporate the parameters
    overrides = [

        # Incorporate parameter exploration params
        AutoDict({
            params_paths[key]: value
            for key, value in param_combination.items()
        }).todict(),  
        
        # Reduce the multimeter interval for disk space
        AutoDict({
            ('network', 'recorder_models', 'my_multimeter', 'nest_params', 'interval'): 50.0
        }).todict(),  

    ]
    print(f'Override trees:\n {overrides}', flush=True)
    
    # Load overriden parameter file
    overriden_tree = denest.load_trees(PARAMS_PATH, *overrides)
    
    # Define output directory for current simulation
    output_dir = MAIN_OUTPUT_DIR/str(param_combination)
    print(output_dir)
    
    # Run simulation
    print(f'Running simulation for param combination: {param_combination}')
    sim = denest.Simulation(overriden_tree, output_dir=output_dir)
    sim.run()
    print(f'done\n\n')

param combination: {'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}
Override trees:
 [{'network': {'neuron_models': {'my_neuron': {'nest_params': {'g_peak_AMPA': 0.1}}}, 'projection_models': {'proj_1_AMPA': {'nest_params': {'weights': 1.0}}, 'proj_2_GABAA': {'nest_params': {'weights': 1.0}}}}}, {'network': {'recorder_models': {'my_multimeter': {'nest_params': {'interval': 50.0}}}}}]


2020-06-28 18:14:24,952 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-28 18:14:24,954 [denest] INFO: Using 2 override tree(s)
2020-06-28 18:14:24,957 [denest] INFO: Finished loading parameter file paths
2020-06-28 18:14:24,958 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']
2020-06-28 18:14:25,072 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-28 18:14:25,077 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-28 18:14:25,081 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-28 18:14:25,110 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-28 18:14:25,113 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-28 18:14:25,121 [denest.simulation] INFO:     Calling `nest.SetKerne

data/output_param_explore/{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}
Running simulation for param combination: {'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}


2020-06-28 18:14:25,149 [denest.simulation] INFO: Finished initializing kernel
2020-06-28 18:14:25,154 [denest.simulation] INFO: Build N=3 session models
2020-06-28 18:14:25,157 [denest.simulation] INFO: Build N=4 sessions
2020-06-28 18:14:25,163 [denest.session] INFO: Creating session "00_warmup"
2020-06-28 18:14:25,165 [denest.utils.validation] INFO: Object `00_warmup`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': [], 'unit_changes': []}
2020-06-28 18:14:25,168 [denest.session] INFO: Creating session "01_3_spikes"
2020-06-28 18:14:25,210 [denest.utils.validation] INFO: Object `01_3_spikes`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': []}
2020-06-28 18:14:25,213 [denest.session] INFO: Creating session "02_2_spikes"
2020-06-28 18:14:25,215 [denest.utils.validation] INFO: Object `02_2_spikes`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_chan

2020-06-28 18:14:26,954 [denest.session] INFO: Finished initializing session

2020-06-28 18:14:26,955 [denest.session] INFO: Running session '02_2_spikes' for 100 ms
2020-06-28 18:14:27,125 [denest.session] INFO: Finished running session
2020-06-28 18:14:27,128 [denest.session] INFO: Session '02_2_spikes' virtual running time: 100 ms
2020-06-28 18:14:27,129 [denest.session] INFO: Session '02_2_spikes' real running time: 0h:00m:00s
2020-06-28 18:14:27,130 [denest.simulation] INFO: Done running session '02_2_spikes'
2020-06-28 18:14:27,132 [denest.simulation] INFO: Running session: '03_3_spikes'...
2020-06-28 18:14:27,134 [denest.session] INFO: Initializing session...
2020-06-28 18:14:27,135 [denest.session] INFO: Setting `origin` flag to `300.0` for all stimulation devices in ``InputLayers`` for session `03_3_spikes`
2020-06-28 18:14:27,141 [denest.utils.validation] INFO: Object `Unit changes dictionary`: params: using default value for optional parameters:
{'change_type': 'constant', '

...

2020-06-28 18:14:42,909 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-28 18:14:42,912 [denest] INFO: Using 2 override tree(s)
2020-06-28 18:14:42,924 [denest] INFO: Finished loading parameter file paths
2020-06-28 18:14:42,929 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']
2020-06-28 18:14:42,992 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-28 18:14:42,993 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-28 18:14:42,994 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-28 18:14:43,006 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-28 18:14:43,009 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-28 18:14:43,017 [denest.simulation] INFO:     Calling `nest.SetKerne

data/output_param_explore/{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}
Running simulation for param combination: {'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}


2020-06-28 18:14:43,131 [denest.simulation] INFO:   Installing external modules...
2020-06-28 18:14:43,141 [denest.simulation] INFO:   Finished installing external modules
2020-06-28 18:14:43,147 [denest.simulation] INFO: Finished initializing kernel
2020-06-28 18:14:43,154 [denest.simulation] INFO: Build N=3 session models
2020-06-28 18:14:43,156 [denest.simulation] INFO: Build N=4 sessions
2020-06-28 18:14:43,160 [denest.session] INFO: Creating session "00_warmup"
2020-06-28 18:14:43,163 [denest.utils.validation] INFO: Object `00_warmup`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': [], 'unit_changes': []}
2020-06-28 18:14:43,166 [denest.session] INFO: Creating session "01_3_spikes"
2020-06-28 18:14:43,169 [denest.utils.validation] INFO: Object `01_3_spikes`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': []}
2020-06-28 18:14:43,170 [denest.session] INFO: Creating session "02_2_spike

2020-06-28 18:14:44,493 [denest.network.layers] INFO: Layer='input_layer', pop='spike_generator': Applying 'constant' change, param='spike_times', from single value')
2020-06-28 18:14:44,718 [denest.session] INFO: Finished initializing session

2020-06-28 18:14:44,720 [denest.session] INFO: Running session '02_2_spikes' for 100 ms
2020-06-28 18:14:44,879 [denest.session] INFO: Finished running session
2020-06-28 18:14:44,880 [denest.session] INFO: Session '02_2_spikes' virtual running time: 100 ms
2020-06-28 18:14:44,882 [denest.session] INFO: Session '02_2_spikes' real running time: 0h:00m:00s
2020-06-28 18:14:44,884 [denest.simulation] INFO: Done running session '02_2_spikes'
2020-06-28 18:14:44,885 [denest.simulation] INFO: Running session: '03_3_spikes'...
2020-06-28 18:14:44,886 [denest.session] INFO: Initializing session...
2020-06-28 18:14:44,889 [denest.session] INFO: Setting `origin` flag to `300.0` for all stimulation devices in ``InputLayers`` for session `03_3_spikes`
2020-

done




In [77]:
!ls {MAIN_OUTPUT_DIR}

{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}
{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 2.0}
{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 1.0}
{'g_peak_AMPA_all_neurons': 0.1, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}
{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 1.0}
{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 1.0, 'weights_proj_2_GABAA': 2.0}
{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 1.0}
{'g_peak_AMPA_all_neurons': 0.2, 'weights_proj_1_AMPA': 2.0, 'weights_proj_2_GABAA': 2.0}
